# 성별 추론 모델 파이프라인
### 사용자의 로그인 전 행동 패턴을 보고 사용자의 성별을 추론하는 머신 러닝 파이프라인 실습입니다. 

## 0) 환경 구성
- 파이프라인을 가동하기 위한 패키지를 설치하고 컴포넌트와 파이프라인을 정의합니다.

In [ ]:
# Cell 0
import kfp, sys
print("KFP:", kfp.__version__, "Python:", sys.version)

from kfp import dsl
from kfp.dsl import Input, Output, Dataset, Model, Metrics, Markdown, HTML

from typing import get_type_hints

def _eval_annotations(fn):
    try:
        # globalns를 넘겨줘야 Dataset/Model 같은 심볼을 올바르게 해석
        fn.__annotations__ = get_type_hints(fn, globalns=fn.__globals__)
    except Exception:
        pass
    return fn

def component2(**kwargs):
    def _decorator(fn):
        fn = _eval_annotations(fn)
        return dsl.component(**kwargs)(fn)
    return _decorator

def pipeline2(**kwargs):
    def _decorator(fn):
        fn = _eval_annotations(fn)
        return dsl.pipeline(**kwargs)(fn)
    return _decorator


## 1) 공통 파라미터 초기화 컴포넌트
- 파이프라인에서 사용하는 공통 파라미터들을 초기화하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 1 — Init & Constants (OutputPath 버전)
from kfp import dsl
from kfp.dsl import OutputPath  # <- 경로로 직접 받음

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = [
    "numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1",
    "joblib==1.4.2", "matplotlib==3.9.0", "pyarrow==17.0.0",
]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell1_init(
    known_cats_json: str,
    production_threshold: float,
    min_prelogin_events: int,
    datasets_dir: str,
    artifact_dir: str,
    params_out_path: OutputPath(str),
):
    import json, unicodedata
    from pathlib import Path

    KNOWN_CATS = [unicodedata.normalize("NFKC", c.strip()) for c in json.loads(known_cats_json)]
    Path(datasets_dir).mkdir(parents=True, exist_ok=True)
    Path(artifact_dir).mkdir(parents=True, exist_ok=True)

    blob = {
        "KNOWN_CATS": KNOWN_CATS,
        "PRODUCTION_THRESHOLD": float(production_threshold),
        "MIN_PRELOGIN_EVENTS": int(min_prelogin_events),
        "DATASETS_DIR": datasets_dir,
        "ARTIFACT_DIR": artifact_dir,
    }
    with open(params_out_path, "w", encoding="utf-8") as f:
        json.dump(blob, f, ensure_ascii=False, indent=2)

print("[done]")

## 2) 로그인 전 구간 필터링 컴포넌트
- PVC에 있는 Join된 원본 데이터를 읽고 로그인 전 구간을 필터링 하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 2 — Read CSV, sort, pre-login slice -> df_out, pre_out, log_txt  (OutputPath 버전)
from kfp import dsl
from kfp.dsl import OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = ["numpy==1.26.4", "pandas==2.2.2", "pyarrow==17.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell2_prelogin_filter(
    input_path: str,
    min_prelogin_events: int,
    df_out: OutputPath("Parquet"),
    pre_out: OutputPath("Parquet"),
    log_txt: OutputPath(str),
):
    import pandas as pd

    df = pd.read_csv(input_path, low_memory=False)
    df["ts"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["session_id", "ts"]).copy()

    sort_cols = ["session_id", "ts"]
    if "event_id" in df.columns:
        sort_cols.append("event_id")
    df = df.sort_values(sort_cols).reset_index(drop=True)

    uid_str = df["user_id"].astype("string").str.strip()
    anon_like = {"", "0", "-1", "None", "none", "NULL", "null", "NaN", "nan"}
    has_uid = uid_str.notna() & ~uid_str.isin(anon_like)
    appeared = has_uid.groupby(df["session_id"]).cummax()
    pre = df.loc[~appeared].copy()

    pre_cnt = pre.groupby("session_id").size()
    valid_sessions = pre_cnt.index[pre_cnt >= int(min_prelogin_events)]
    pre = pre[pre["session_id"].isin(valid_sessions)].copy()

    df.to_parquet(df_out, index=False)
    pre.to_parquet(pre_out, index=False)

    with open(log_txt, "w", encoding="utf-8") as f:
        f.write(f"원본 행 수: {len(df)}\n")
        f.write(f"로그인 전 로그 필터 후 행 수: {len(pre)}\n")
        f.write(f"로그인 전 유일 세션 수: {pre['session_id'].nunique()}\n")

print("[done]")

## 3) 피처 집계 & 성별 라벨링 컴포넌트
- 로그인 전 로그를 세션 단위로 집계하고 성별 라벨을 붙여 학습용 데이터를 생성하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 3 — Aggregate numeric features + labels -> Xnum_out, y_out, keep_sessions_out, counts_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = ["numpy==1.26.4", "pandas==2.2.2", "pyarrow==17.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell3_aggregate_and_label(
    df_in: InputPath("Parquet"),
    pre_in: InputPath("Parquet"),
    Xnum_out: OutputPath("Parquet"),
    y_out: OutputPath("CSV"),
    keep_sessions_out: OutputPath(str),
    counts_txt: OutputPath(str),
):
    import pandas as pd

    df = pd.read_parquet(df_in)
    pre = pd.read_parquet(pre_in)

    df_gender = df.copy()
    df_gender["gender_norm"] = (
        df_gender["gender"].astype("string").str.strip().str.upper().replace({"FEMALE":"F", "MALE":"M"})
    )
    lab_full = (
        df_gender[df_gender["gender_norm"].isin(["M","F"])]
          .groupby("session_id")["gender_norm"].agg(lambda s: s.iloc[0])
    )

    agg_num = pre.groupby("session_id").agg(
        n_events=("session_id", "size"),
        search_count_sum=("search_count", "sum"),
        cart_item_count_sum=("cart_item_count", "sum"),
        page_depth_mean=("page_depth", "mean"),
        last_elapsed_mean=("last_action_elapsed", "mean"),
        unique_pages=("current_state", "nunique"),
        unique_categories=("resolved_category", "nunique"),
    ).fillna(0.0)

    first_ts = pd.to_datetime(pre.groupby("session_id")["ts"].min(), errors="coerce")
    agg_num["start_hour"] = first_ts.dt.hour
    agg_num["start_weekday"] = first_ts.dt.weekday

    keep_sessions = agg_num.index.intersection(lab_full.index)
    X_num = agg_num.loc[keep_sessions].copy()
    y = lab_full.loc[keep_sessions].rename("gender").copy()

    X_num.to_parquet(Xnum_out, index=True)
    y.to_frame().to_csv(y_out, index=True, header=True)
    pd.Series(keep_sessions, name="session_id").to_csv(keep_sessions_out, index=False, header=False)

    with open(counts_txt, "w", encoding="utf-8") as f:
        f.write(f"로그인 완료한 세션 수: {len(keep_sessions)}\n")
        f.write("성별 분포:\n\n")
        f.write(y.value_counts(dropna=False).to_string())

print("[done]")

## 4) 카운트 & 파생 컬럼 생성 컴포넌트
- 카테고리 등장 횟수를 저장하는 카운트 컬럼, 횟수를 비율로 치환하는 파생 컬럼을 생성하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 4 — Category count/prop/log features -> cat_cnt_out, cat_prop_out, cat_log_out, cats_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = ["numpy==1.26.4", "pandas==2.2.2", "pyarrow==17.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell4_category_features(
    pre_in: InputPath("Parquet"),
    Xnum_in: InputPath("Parquet"),
    known_cats_json: str,
    cat_cnt_out: OutputPath("Parquet"),
    cat_prop_out: OutputPath("Parquet"),
    cat_log_out: OutputPath("Parquet"),
    cats_txt: OutputPath(str),
):
    import pandas as pd, numpy as np, json, unicodedata

    pre = pd.read_parquet(pre_in)
    X_num = pd.read_parquet(Xnum_in)

    KNOWN_CATS_NORM = [unicodedata.normalize("NFKC", c.strip()) for c in json.loads(known_cats_json)]
    norm_to_orig = {unicodedata.normalize("NFKC", c.strip()): c for c in json.loads(known_cats_json)}

    pre_cat_norm = pre["resolved_category"].astype("string").str.strip().apply(
        lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x
    )
    mask = pre_cat_norm.isin(KNOWN_CATS_NORM)
    pre_kept = pre[mask].copy()
    pre_kept["cat_norm"] = pre_cat_norm[mask].values
    pre_kept["one"] = 1

    cat_cnt = pre_kept.pivot_table(index="session_id", columns="cat_norm", values="one", aggfunc="sum", fill_value=0)
    cat_cnt = cat_cnt.reindex(columns=KNOWN_CATS_NORM, fill_value=0)
    cat_cnt.columns = [norm_to_orig[c] for c in cat_cnt.columns]
    cat_cnt.columns = [f"cat_cnt::{c}" for c in cat_cnt.columns]
    cat_cnt = cat_cnt.reindex(X_num.index).fillna(0).astype(int)

    cat_prop = cat_cnt.div(X_num["n_events"].replace(0, 1), axis=0)
    cat_prop.columns = [c.replace("cat_cnt::", "cat_prop::") for c in cat_cnt.columns]

    cat_log = np.log1p(cat_cnt)
    cat_log.columns = [c.replace("cat_cnt::", "cat_log::") for c in cat_cnt.columns]

    cat_cnt.to_parquet(cat_cnt_out, index=True)
    cat_prop.to_parquet(cat_prop_out, index=True)
    cat_log.to_parquet(cat_log_out, index=True)

    with open(cats_txt, "w", encoding="utf-8") as f:
        f.write("카테고리 카운트 컬럼:\n")
        f.write(", ".join(list(cat_cnt.columns)) + "\n\n")
        f.write("카테고리 파생 컬럼 예시:\n")
        f.write(", ".join(list(cat_prop.columns[:3]) + list(cat_log.columns[:3])))

print("[done]")

## 5) 최종 학습 데이터 셋 생성 컴포넌트
- 카운트 & 파생 컬럼과 라벨을 합쳐 최종 학습용 데이터 셋을 저장하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 5 — Join all features + y -> dataset_csv_out, dataset_pq_out, head_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = ["numpy==1.26.4", "pandas==2.2.2", "pyarrow==17.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell5_build_dataset(
    Xnum_in: InputPath("Parquet"),
    cat_cnt_in: InputPath("Parquet"),
    cat_prop_in: InputPath("Parquet"),
    cat_log_in: InputPath("Parquet"),
    y_in: InputPath("CSV"),
    dataset_csv_out: OutputPath("CSV"),
    dataset_pq_out: OutputPath("Parquet"),
    head_txt: OutputPath(str),
):
    import pandas as pd

    X_num = pd.read_parquet(Xnum_in)
    cat_cnt = pd.read_parquet(cat_cnt_in)
    cat_prop = pd.read_parquet(cat_prop_in)
    cat_log = pd.read_parquet(cat_log_in)
    y = pd.read_csv(y_in, index_col=0)["gender"]

    X = (X_num.join(cat_cnt, how="left")
              .join(cat_prop, how="left")
              .join(cat_log,  how="left")).fillna(0)

    X_ = X.copy(); X_.index.name = "session_id"
    dataset = X_.join(y).reset_index()

    dataset.to_csv(dataset_csv_out, index=False)
    dataset.to_parquet(dataset_pq_out, index=False)

    with open(head_txt, "w", encoding="utf-8") as f:
        f.write(f"dataset shape: {dataset.shape}\n\n")
        f.write(dataset.head(10).to_string(index=False))

print("[done]")

## 6) 데이터 셋 분할 컴포넌트
- 최종 데이터 셋을 학습용 / 검증용 데이터 셋으로 분할하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 6 — Stratified split -> Xtr_out, Xva_out, ytr_out, yva_out, split_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = ["numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1", "pyarrow==17.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell6_split(
    dataset_pq_in: InputPath("Parquet"),
    random_state: int,
    test_size: float,
    Xtr_out: OutputPath("Parquet"),
    Xva_out: OutputPath("Parquet"),
    ytr_out: OutputPath("CSV"),
    yva_out: OutputPath("CSV"),
    split_txt: OutputPath(str),
):
    import pandas as pd
    from sklearn.model_selection import StratifiedShuffleSplit

    dataset = pd.read_parquet(dataset_pq_in)
    dataset["gender"] = (
        dataset["gender"].astype("string").str.strip().str.upper().replace({"FEMALE":"F","MALE":"M"})
    )

    non_feature = {"session_id","gender"}
    feature_cols = [c for c in dataset.columns if c not in non_feature]
    X = dataset[feature_cols].copy()
    y_bin = dataset["gender"].map({"F":0, "M":1}).astype(int)

    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=int(random_state))
    train_idx, valid_idx = next(sss.split(X, y_bin))

    # ★ 인덱스 보존(후속 셀에서 매핑에 필요)
    X.iloc[train_idx].to_parquet(Xtr_out, index=True)
    X.iloc[valid_idx].to_parquet(Xva_out, index=True)
    y_bin.iloc[train_idx].to_frame("y").to_csv(ytr_out, index=False)
    y_bin.iloc[valid_idx].to_frame("y").to_csv(yva_out, index=False)

    with open(split_txt, "w", encoding="utf-8") as f:
        f.write(f"Train: {X.iloc[train_idx].shape}  Valid: {X.iloc[valid_idx].shape}\n")
        f.write("Label dist (train):\n")
        f.write(y_bin.iloc[train_idx].value_counts(normalize=True).rename({0:"F",1:"M"}).to_string() + "\n\n")
        f.write("Label dist (valid):\n")
        f.write(y_bin.iloc[valid_idx].value_counts(normalize=True).rename({0:"F",1:"M"}).to_string())

print("[done]")

## 7) Logistic 모델 학습 컴포넌트
- Logistic 모델을 학습하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 7 — Train base Logistic & Calibrate -> cal_model_out, train_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = [
    "numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1",
    "joblib==1.4.2", "pyarrow==17.0.0"
]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell7_train_calibrate(
    Xtr_in: InputPath("Parquet"),
    ytr_in: InputPath("CSV"),
    calib_method: str,
    calib_cv: int,
    cal_model_out: OutputPath("Model"),
    train_txt: OutputPath(str),
):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.calibration import CalibratedClassifierCV
    from joblib import dump

    X_train = pd.read_parquet(Xtr_in)
    y_train = pd.read_csv(ytr_in)["y"].astype(int)

    log_best = Pipeline([
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
        ("clf", LogisticRegression(solver="saga", penalty="l2", C=1.0, max_iter=5000, random_state=42)),
    ])
    log_best.fit(X_train, y_train)

    try:
        cal_log = CalibratedClassifierCV(estimator=log_best, method=calib_method, cv=int(calib_cv))
    except TypeError:
        cal_log = CalibratedClassifierCV(base_estimator=log_best, method=calib_method, cv=int(calib_cv))

    cal_log.fit(X_train, y_train)
    dump(cal_log, cal_model_out)

    with open(train_txt, "w", encoding="utf-8") as f:
        f.write(f"[Calibration] done: method={calib_method}, cv={calib_cv}\n")

print("[done]")

## 8) 모델 검증 및 평가 컴포넌트
- 학습한 모델을 검증하여 평가하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 8 — Evaluate @ production threshold + age metrics -> proba_out, pred_out, age_metrics_out, eval_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = [
    "numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1",
    "joblib==1.4.2", "pyarrow==17.0.0"
]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell8_eval_prod_threshold(
    cal_model_in: InputPath("Model"),
    Xva_in: InputPath("Parquet"),
    yva_in: InputPath("CSV"),
    dataset_csv_in: InputPath("CSV"),
    production_threshold: float,
    proba_out: OutputPath("CSV"),
    pred_out: OutputPath("CSV"),
    age_metrics_out: OutputPath("CSV"),
    eval_txt: OutputPath(str),
):
    import pandas as pd, numpy as np
    from joblib import load
    from sklearn.metrics import classification_report, confusion_matrix, f1_score

    cal_log = load(cal_model_in)
    X_valid = pd.read_parquet(Xva_in)
    y_valid = pd.read_csv(yva_in)["y"].astype(int)

    proba_v = cal_log.predict_proba(X_valid)[:, 1]
    pred_bin = (proba_v >= float(production_threshold)).astype(int)

    pd.DataFrame({"proba": proba_v}).to_csv(proba_out, index=False)
    pd.DataFrame({"pred":  pred_bin}).to_csv(pred_out, index=False)

    macro_f1 = f1_score(y_valid, pred_bin, average="macro")
    acc = (y_valid.values == pred_bin).mean()

    lines = []
    lines.append(f"=== LogisticCal (prod={production_threshold:.3f}) ===")
    lines.append(classification_report(y_valid, pred_bin, target_names=["F","M"], digits=4))
    lines.append("Confusion matrix [rows=true F,M | cols=pred F,M]:")
    lines.append(str(confusion_matrix(y_valid, pred_bin)))
    lines.append(f"[LogisticCal (prod@{production_threshold:.3f})] Macro-F1 : {macro_f1:.4f} / Accuracy : {acc:.4f}")

    dataset = pd.read_csv(dataset_csv_in)
    sess_ids_valid = dataset.iloc[X_valid.index]["session_id"].values
    out_df = pd.DataFrame({
        "session_id": sess_ids_valid,
        "y_true": y_valid.values,
        f"y_pred@{production_threshold:.3f}": pred_bin,
        "proba_cal": proba_v,
    })

    extra_cols = [c for c in ["age", "age_group"] if c in dataset.columns]
    if extra_cols:
        out_df = out_df.merge(
            dataset[["session_id"] + extra_cols].drop_duplicates("session_id"),
            on="session_id", how="left"
        )

    metrics_df = out_df.copy()
    metrics_df["age_group"] = metrics_df.get("age_group", pd.Series([np.nan]*len(metrics_df))).astype("string").fillna("unknown")

    rows = []
    pred_col = [c for c in metrics_df.columns if c.startswith("y_pred@")][0]
    for g, sub in metrics_df.groupby("age_group", dropna=False):
        y_t = sub["y_true"].astype(int).to_numpy()
        y_h = sub[pred_col].astype(int).to_numpy()
        acc_g = float((y_t == y_h).mean())
        f1_g  = float(f1_score(y_t, y_h, average="macro")) if len(np.unique(y_t)) > 1 else float("nan")
        rows.append({"age_group": str(g), "acc": acc_g, "macro_f1": f1_g})

    pd.DataFrame(rows).to_csv(age_metrics_out, index=False)
    with open(eval_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

print("[done]")

## 9) Katib 최적 하이퍼파라미터 로드 컴포넌트
- Katib에서 최적 하이퍼파라미터를 로드하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 9 — Fetch Katib best params -> katib_params_json_out, katib_df_csv_out
from kfp import dsl
from kfp.dsl import OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_K8S = ["numpy==1.26.4", "pandas==2.2.2", "pyarrow==17.0.0", "kubernetes==29.0.0"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_K8S)
def cell9_fetch_katib(
    katib_namespace: str,
    katib_experiment: str,
    katib_params_json_out: OutputPath(str),
    katib_df_csv_out: OutputPath("CSV"),
):
    import pandas as pd, json
    from kubernetes import client, config

    try:
        config.load_incluster_config()
    except Exception:
        config.load_kube_config()

    api = client.CustomObjectsApi()
    obj = api.get_namespaced_custom_object(
        group="kubeflow.org", version="v1beta1", plural="experiments",
        namespace=katib_namespace, name=katib_experiment
    )

    status = obj.get("status", {})
    trial = status.get("currentOptimalTrial") or status.get("optimalTrial") or status.get("bestTrial") or {}
    params = trial.get("parameterAssignments", [])
    if not params:
        params = []

    with open(katib_params_json_out, "w", encoding="utf-8") as f:
        json.dump(params, f, ensure_ascii=False, indent=2)

    pd.DataFrame(params).to_csv(katib_df_csv_out, index=False)

print("[done]")

## 10) 모델 재학습 컴포넌트
- katib에서 찾은 최적 하이퍼파라미터를 적용해 모델을 재학습하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 10 — Train with Katib HPs (+optional rebuild) -> cal_model_out, Xva_out, yva_out, dataset_out, train_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = [
    "numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1",
    "joblib==1.4.2", "pyarrow==17.0.0"
]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell10_train_with_katib(
    katib_params_json_in: str,         
    input_path: str,
    known_cats_json: str,
    base_min_prelogin_events: int,
    Xva_prev_in: InputPath("Parquet"),
    yva_prev_in: InputPath("CSV"),
    dataset_prev_in: InputPath("CSV"),
    cal_model_out: OutputPath("Model"),
    Xva_out: OutputPath("Parquet"),
    yva_out: OutputPath("CSV"),
    dataset_out: OutputPath("CSV"),
    train_txt: OutputPath(str),
):
    import json, os, unicodedata, numpy as np, pandas as pd
    from sklearn.preprocessing import StandardScaler, RobustScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.calibration import CalibratedClassifierCV
    from sklearn.model_selection import StratifiedShuffleSplit
    from joblib import dump

    # --- Katib 파라미터 파싱 (문자열 or 경로 둘 다 안전 처리) ---
    raw = katib_params_json_in or "[]"
    try:
        if raw.strip().startswith("["):
            params = json.loads(raw)
        elif os.path.exists(raw):
            params = json.load(open(raw, "r", encoding="utf-8"))
        else:
            params = []
    except Exception:
        params = []
    K = {d["name"]: str(d["value"]).strip() for d in params} if params else {}

    def _get_str(m, k, default=None):
        v = m.get(k, default); return None if v is None else str(v).strip()
    def _get_float(m, k, default=None):
        v = m.get(k, None)
        try: return float(v)
        except: return default
    def _get_int(m, k, default=None):
        v = m.get(k, None)
        try: return int(float(v))
        except: return default

    penalty   = (_get_str(K, "penalty", "l2") or "l2").lower()
    C         = _get_float(K, "C", 1.0)
    l1_ratio  = _get_float(K, "l1_ratio", None)
    scaler_nm = (_get_str(K, "scaler", "standard") or "standard").lower()
    cw_nm     = _get_str(K, "class_weight", None)
    class_weight = None if (cw_nm is None or cw_nm.lower()=="none") else ("balanced" if cw_nm.lower()=="balanced" else cw_nm)

    calib_method = (_get_str(K, "calib_method", "isotonic") or "isotonic").lower()
    if calib_method not in {"isotonic","sigmoid"}: calib_method = "isotonic"
    calib_cv   = _get_int(K, "calib_cv", 5)

    t_lo    = _get_float(K, "tune_t_lo", 0.2)
    t_hi    = _get_float(K, "tune_t_hi", 0.8)
    t_steps = _get_int (K, "tune_t_steps", 61)
    if (t_lo is not None) and (t_hi is not None) and (t_lo > t_hi):
        t_lo, t_hi = t_hi, t_lo

    min_prelogin_events_k = _get_int(K, "min_prelogin_events", None)
    need_rebuild = (min_prelogin_events_k is not None) and (int(min_prelogin_events_k) != int(base_min_prelogin_events))

    def _norm(series: pd.Series) -> pd.Series:
        s = series.astype("string").str.strip()
        return s.apply(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)

    def _build_dataset_from_raw(input_path: str, min_events: int, known_cats_json: str):
        df = pd.read_csv(input_path, low_memory=False)
        df["ts"] = pd.to_datetime(df["timestamp"], errors="coerce")
        df = df.dropna(subset=["session_id","ts"]).copy()
        sort_cols = ["session_id","ts"] + (["event_id"] if "event_id" in df.columns else [])
        df = df.sort_values(sort_cols).reset_index(drop=True)

        uid_str  = df["user_id"].astype("string").str.strip()
        anon_like = {"", "0", "-1", "None", "none", "NULL", "null", "NaN", "nan"}
        has_uid = uid_str.notna() & ~uid_str.isin(anon_like)
        appeared = has_uid.groupby(df["session_id"]).cummax()
        pre = df.loc[~appeared].copy()

        pre_cnt = pre.groupby("session_id").size()
        valid_sessions = pre_cnt.index[pre_cnt >= int(min_events)]
        pre = pre[pre["session_id"].isin(valid_sessions)].copy()

        df_gender = df.copy()
        df_gender["gender_norm"] = (
            df_gender.get("gender", pd.Series(index=df_gender.index, dtype="object")).astype("string").str.strip().str.upper().replace({"FEMALE":"F","MALE":"M"})
        )
        lab_full = df_gender[df_gender["gender_norm"].isin(["M","F"])].groupby("session_id")["gender_norm"].agg(lambda s: s.iloc[0])

        agg_num = pre.groupby("session_id").agg(
            n_events=("session_id","size"),
            search_count_sum=("search_count","sum") if "search_count" in pre.columns else ("session_id","size"),
            cart_item_count_sum=("cart_item_count","sum") if "cart_item_count" in pre.columns else ("session_id","size"),
            page_depth_mean=("page_depth","mean") if "page_depth" in pre.columns else ("session_id","size"),
            last_elapsed_mean=("last_action_elapsed","mean") if "last_action_elapsed" in pre.columns else ("session_id","size"),
            unique_pages=("current_state","nunique") if "current_state" in pre.columns else ("session_id","size"),
            unique_categories=("resolved_category","nunique") if "resolved_category" in pre.columns else ("session_id","size"),
        ).fillna(0.0)
        first_ts = pre.groupby("session_id")["ts"].min()
        agg_num["start_hour"] = first_ts.dt.hour
        agg_num["start_weekday"] = first_ts.dt.weekday

        import json
        KNOWN_CATS = [unicodedata.normalize("NFKC", c.strip()) for c in json.loads(known_cats_json)]
        norm_to_orig = {unicodedata.normalize("NFKC", c.strip()): c for c in json.loads(known_cats_json)}
        pre_cat_norm = _norm(pre.get("resolved_category", pd.Series(index=pre.index, dtype="object")))
        mask = pre_cat_norm.isin(KNOWN_CATS)
        pre_kept = pre[mask].copy()
        pre_kept["cat_norm"] = pre_cat_norm[mask].values
        pre_kept["one"] = 1
        cat_cnt = pre_kept.pivot_table(index="session_id", columns="cat_norm", values="one", aggfunc="sum", fill_value=0)
        cat_cnt = cat_cnt.reindex(columns=KNOWN_CATS, fill_value=0)
        cat_cnt.columns = [norm_to_orig[c] for c in cat_cnt.columns]
        cat_cnt.columns = [f"cat_cnt::{c}" for c in cat_cnt.columns]
        cat_cnt = cat_cnt.reindex(agg_num.index).fillna(0).astype(int)

        cat_prop = cat_cnt.div(agg_num["n_events"].replace(0,1), axis=0)
        cat_prop.columns = [c.replace("cat_cnt::","cat_prop::") for c in cat_cnt.columns]
        cat_log = np.log1p(cat_cnt)
        cat_log.columns = [c.replace("cat_cnt::","cat_log::") for c in cat_cnt.columns]

        X = (agg_num.join(cat_cnt, how="left").join(cat_prop, how="left").join(cat_log, how="left")).fillna(0)
        keep_sessions = X.index.intersection(lab_full.index)
        X = X.loc[keep_sessions].copy()
        y = lab_full.loc[keep_sessions].rename("gender").copy()

        X_ = X.copy(); X_.index.name = "session_id"
        dataset = X_.join(y).reset_index()
        dataset["gender"] = dataset["gender"].astype("string").str.strip().str.upper().replace({"FEMALE":"F","MALE":"M"})
        return dataset

    if need_rebuild:
        dataset = _build_dataset_from_raw(input_path, int(min_prelogin_events_k), known_cats_json)
        non_feature = {"session_id","gender"}
        feature_cols = [c for c in dataset.columns if c not in non_feature]
        X_all = dataset[feature_cols].copy()
        y_bin = dataset["gender"].map({"F":0,"M":1}).astype(int)
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        _, va_idx = next(sss.split(X_all, y_bin))
        X_valid, y_valid = X_all.iloc[va_idx], y_bin.iloc[va_idx]
    else:
        dataset = pd.read_csv(dataset_prev_in)
        X_valid = pd.read_parquet(Xva_prev_in)
        y_valid = pd.read_csv(yva_prev_in)["y"].astype(int)

    ScalerClass  = StandardScaler if scaler_nm == "standard" else RobustScaler
    clf_kwargs = dict(solver="saga", penalty=penalty, C=float(C), max_iter=5000, random_state=42)
    if penalty == "elasticnet" and l1_ratio is not None:
        clf_kwargs["l1_ratio"] = float(l1_ratio)
    if class_weight is not None:
        clf_kwargs["class_weight"] = class_weight

    non_feature = {"session_id","gender"}
    feature_cols = [c for c in dataset.columns if c not in non_feature]
    X_all = dataset[feature_cols].copy()
    y_all = dataset["gender"].map({"F":0,"M":1}).astype(int)

    log_katib = Pipeline([
        ("scaler", ScalerClass(with_mean=True, with_std=True) if ScalerClass is StandardScaler else ScalerClass(with_centering=True, with_scaling=True)),
        ("clf", LogisticRegression(**clf_kwargs)),
    ])
    log_katib.fit(X_all, y_all)

    try:
        cal_katib = CalibratedClassifierCV(estimator=log_katib, method=calib_method, cv=int(calib_cv))
    except TypeError:
        cal_katib = CalibratedClassifierCV(base_estimator=log_katib, method=calib_method, cv=int(calib_cv))
    cal_katib.fit(X_all, y_all)

    dump(cal_katib, cal_model_out)
    X_valid.to_parquet(Xva_out, index=False)
    y_valid.to_frame("y").to_csv(yva_out, index=False)
    dataset.to_csv(dataset_out, index=False)

    with open(train_txt, "w", encoding="utf-8") as f:
        f.write(
            f"[OK] 모델 학습 완료 → min_prelogin_events={min_prelogin_events_k if need_rebuild else base_min_prelogin_events}, "
            f"scaler={scaler_nm}, penalty={penalty}, C={C}, class_weight={class_weight}, "
            f"calibration=({calib_method}, cv={calib_cv})\n"
        )

print("[done]")

## 11) 모델 검증 및 평가 컴포넌트
- Katib 최적 하이퍼파라미터로 학습한 모델을 검증 및 평가하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 11 — Tune threshold + save meta -> best_t_txt, proba_csv_out, age_metrics_out, meta_json_out, eval_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_COMMON = [
    "numpy==1.26.4", "pandas==2.2.2", "scikit-learn==1.5.1",
    "joblib==1.4.2", "pyarrow==17.0.0"
]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_COMMON)
def cell11_tune_threshold_and_save(
    cal_model_in: InputPath("Model"),
    Xva_in: InputPath("Parquet"),
    yva_in: InputPath("CSV"),     
    dataset_in: InputPath("CSV"), 
    df_in: InputPath("Parquet"),
    t_lo: float,
    t_hi: float,
    t_steps: int,
    artifact_dir_hint: str,
    best_t_txt: OutputPath(str),
    proba_csv_out: OutputPath("CSV"),
    age_metrics_out: OutputPath("CSV"), 
    meta_json_out: OutputPath(str),
    eval_txt: OutputPath(str),
):
    import os, json, numpy as np, pandas as pd
    from joblib import load, dump
    from sklearn.metrics import f1_score

    model = load(cal_model_in)
    X_valid = pd.read_parquet(Xva_in)
    y_valid = pd.read_csv(yva_in)["y"].astype(int)
    dataset = pd.read_csv(dataset_in)
    df = pd.read_parquet(df_in)

    lo, hi, n = float(t_lo), float(t_hi), int(t_steps)
    ths = np.linspace(lo, hi, n)
    proba_k = model.predict_proba(X_valid)[:, 1]
    best_t, best_f1 = 0.5, -1.0
    for t in ths:
        f1 = f1_score(y_valid, (proba_k >= t).astype(int), average="macro")
        if f1 > best_f1:
            best_t, best_f1 = float(t), float(f1)

    with open(best_t_txt, "w", encoding="utf-8") as f:
        f.write(f"{best_t}")
    pd.DataFrame({"proba": proba_k}).to_csv(proba_csv_out, index=False)

    sess_ids = dataset.loc[X_valid.index, "session_id"].to_numpy() if "session_id" in dataset.columns else np.arange(len(X_valid))
    pred_bin = (proba_k >= best_t).astype(int)
    pred_col = f"y_pred@{best_t:.3f}"
    out_df = pd.DataFrame({
        "session_id": sess_ids,
        "y_true": y_valid.values,
        pred_col: pred_bin,
        "proba_cal": proba_k,
    })

    extra_cols = [c for c in ["age", "age_group"] if c in dataset.columns]
    if extra_cols:
        out_df = out_df.merge(
            dataset[["session_id"] + extra_cols].drop_duplicates("session_id"),
            on="session_id", how="left"
        )

    if "age_group" not in out_df.columns and "age" in df.columns:
        uid_str  = df.get("user_id", pd.Series([""] * len(df))).astype("string").str.strip()
        anon_like = {"", "0", "-1", "None", "none", "NULL", "null", "NaN", "nan"}
        has_uid = uid_str.notna() & ~uid_str.isin(anon_like)
        appeared_local = has_uid.groupby(df["session_id"]).cummax()
        post = df.loc[appeared_local].copy()
        age_post_by_sess = (
            pd.to_numeric(post["age"], errors="coerce")
              .groupby(post["session_id"])
              .apply(lambda s: s.dropna().iloc[0] if len(s.dropna()) else np.nan)
              .rename("age")
        )
        age_map = age_post_by_sess.to_frame()

        def _to_age_bucket(x):
            if pd.isna(x): return np.nan
            x = float(x)
            return "young" if x < 25 else ("middle" if x < 50 else "old")

        age_map["age_group"] = age_map["age"].apply(_to_age_bucket)
        out_df = out_df.merge(age_map.reset_index(), on="session_id", how="left")

    out_df["age_group"] = out_df.get("age_group").astype("string").fillna("unknown")
    rows = []
    for g, sub in out_df.groupby("age_group", dropna=False):
        y_true_g = sub["y_true"].astype(int).to_numpy()
        y_pred_g = sub[pred_col].astype(int).to_numpy()
        acc = float((y_true_g == y_pred_g).mean())
        f1  = float(f1_score(y_true_g, y_pred_g, average="macro")) if len(np.unique(y_true_g)) > 1 else float("nan")
        rows.append({"age_group": str(g), "acc": acc, "macro_f1": f1})

    pd.DataFrame(rows).sort_values("age_group").to_csv(age_metrics_out, index=False)

    meta = {
        "source": "katib",
        "best_params": None,
        "best_cv_f1_macro": None,
        "production_threshold": float(best_t),
        "calibration": None,
        "threshold_tuning": {"lo": float(lo), "hi": float(hi), "steps": int(n)},
    }
    with open(meta_json_out, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    try:
        os.makedirs(artifact_dir_hint, exist_ok=True)
        dump(model, os.path.join(artifact_dir_hint, "model_calibrated.joblib"))
    except Exception:
        pass

    with open(eval_txt, "w", encoding="utf-8") as f:
        f.write(f"[Katib HP] Best macro-F1 in [{lo:.3f},{hi:.3f}] steps={n}: t={best_t:.3f}, F1={best_f1:.4f}\n")

print("[done]")

## 12) Object Storage에 모델 저장 컴포넌트
- 검증 및 평가된 모델을 Object Storage에 저장하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 12 — Upload trained model to Kakao Object Storage -> uploaded_uri_out, log_txt
from kfp import dsl
from kfp.dsl import InputPath, OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_S3 = ["boto3==1.34.162", "botocore==1.34.162"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_S3)
def cell12_upload_model_to_kakao(
    # --- 필수(기본값 없음) ---
    cal_model_in: InputPath("Model"),
    endpoint: str,
    region: str,
    bucket: str,
    access_key: str,           
    secret_key: str,              
    uploaded_uri_out: OutputPath(str),
    log_txt: OutputPath(str),

    # --- 선택(기본값 있음) ---
    prefix: str = "gender_predict_pipeline",
    object_name: str = "model.joblib",
    public_read: bool = False,
):
    import os, time, hashlib
    import boto3

    # 1) boto3 클라이언트
    ak = access_key or os.getenv("AWS_ACCESS_KEY_ID") or os.getenv("KAKAO_S3_ACCESS_KEY_ID")
    sk = secret_key or os.getenv("AWS_SECRET_ACCESS_KEY") or os.getenv("KAKAO_S3_SECRET_ACCESS_KEY")
    if not ak or not sk:
        raise RuntimeError("Missing credentials: provide access_key/secret_key or set env vars.")

    s3 = boto3.client(
        "s3",
        endpoint_url=endpoint,
        region_name=region,
        aws_access_key_id=ak,
        aws_secret_access_key=sk,
    )

    # 2) 버킷 존재 보장
    try:
        s3.head_bucket(Bucket=bucket)
    except Exception:
        try:
            s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={"LocationConstraint": region})
        except Exception as e2:
            code = getattr(e2, "response", {}).get("Error", {}).get("Code")
            if code not in ("BucketAlreadyOwnedByYou", "BucketAlreadyExists"):
                raise

    # 3) 업로드 키 결정
    key_prefix = (prefix.strip("/") + "/") if prefix and prefix.strip() else ""
    if not object_name:
        object_name = f"model_calibrated_{time.strftime('%Y%m%d-%H%M%S')}.joblib"
    key = f"{key_prefix}{object_name}"

    # 4) 업로드
    extra_args = {"ContentType": "application/octet-stream"}
    if public_read:
        extra_args["ACL"] = "public-read"
    s3.upload_file(cal_model_in, bucket, key, ExtraArgs=extra_args)

    # 5) 무결성/로그
    h = hashlib.md5()
    with open(cal_model_in, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    md5 = h.hexdigest()
    s3_uri = f"s3://{bucket}/{key}"

    with open(uploaded_uri_out, "w", encoding="utf-8") as f:
        f.write(s3_uri)
    with open(log_txt, "w", encoding="utf-8") as f:
        f.write(
            "=== Kakao Object Storage Upload ===\n"
            f"Endpoint: {endpoint}\nRegion  : {region}\n"
            f"Bucket  : {bucket}\nPrefix  : {key_prefix}\n"
            f"Object  : {object_name}\nMD5     : {md5}\n"
            f"URI     : {s3_uri}\n"
        )

print("[done]")

## 13) KServe 모델 배포 컴포넌트
- Object Storage에 저장된 모델을 KServe InferenceService로 배포하는 컴포넌트를 정의합니다.

In [ ]:
# Cell 13 — Deploy with KServe (from Kakao Object Storage) -> isvc_name_out, isvc_yaml_out, isvc_url_out, log_txt
from kfp import dsl
from kfp.dsl import OutputPath

BASE_IMAGE = "python:3.11-slim"
PKGS_K8S = ["kubernetes==29.0.0", "pyyaml==6.0.1"]

@component2(base_image=BASE_IMAGE, packages_to_install=PKGS_K8S)
def cell13_deploy_kserve(
    # === Required params ===
    namespace: str,               # 배포 네임스페이스
    isvc_name: str,               # InferenceService 이름
    bucket: str,                  # S3 버킷명 (예: models)
    prefix: str,                  # 경로/폴더 (예: gender_predict_pipeline)
    s3_endpoint: str,             # https://objectstorage.kr-central-2.kakaocloud.com
    s3_region: str,               # kr-central-2
    aws_access_key_id: str,       # 액세스 키
    aws_secret_access_key: str,   # 시크릿 키

    # === Outputs (기본값 있는 인자보다 먼저 둬야 파이썬 시그니처 오류가 안 남) ===
    isvc_name_out: OutputPath(str),
    isvc_yaml_out: OutputPath(str),
    isvc_url_out: OutputPath(str),
    log_txt: OutputPath(str),

    # === Optional params ===
    min_replicas: int = 1,
    max_replicas: int = 1,
    use_legacy_spec: bool = False,    # False: predictor.model, True: predictor.sklearn
    append_model_filename: bool = False,  # True면 /model.joblib 까지 붙임
):
    import time, yaml
    from kubernetes import client, config
    from kubernetes.client import ApiException

    # ---- sanitize & compose storageUri ----
    bucket = bucket.strip().strip("/")
    prefix = prefix.strip().strip("/")
    s3_endpoint = s3_endpoint.strip()
    s3_host = s3_endpoint.replace("https://", "").replace("http://", "").strip("/")
    storage_uri = f"s3://{bucket}/{prefix}"
    if append_model_filename:
        storage_uri = storage_uri + "/model.joblib"

    # ---- K8s 연결 ----
    try:
        config.load_incluster_config()
    except Exception:
        config.load_kube_config()
    core = client.CoreV1Api()
    crd = client.CustomObjectsApi()

    # ---- S3 Secret 생성/갱신 (KServe 크레덴셜 인젝터용 어노테이션 포함) ----
    secret_name = f"{isvc_name}-s3"
    secret_body = client.V1Secret(
        metadata=client.V1ObjectMeta(
            name=secret_name,
            annotations={
                "serving.kserve.io/s3-endpoint":  s3_host,
                "serving.kserve.io/s3-usehttps":  "1",
                "serving.kserve.io/s3-verifyssl": "1",
                "serving.kserve.io/s3-region":    s3_region,
            },
        ),
        type="Opaque",
        string_data={
            "AWS_ACCESS_KEY_ID":     aws_access_key_id,
            "AWS_SECRET_ACCESS_KEY": aws_secret_access_key,
        },
    )
    try:
        core.create_namespaced_secret(namespace, secret_body)
    except ApiException as e:
        if e.status == 409:
            core.patch_namespaced_secret(secret_name, namespace, secret_body)
        else:
            raise

    # ---- ServiceAccount 생성/갱신 (Secret 연결) ----
    sa_name = f"{isvc_name}-sa"
    sa_body = client.V1ServiceAccount(
        metadata=client.V1ObjectMeta(name=sa_name),
        secrets=[client.V1ObjectReference(name=secret_name)],
    )
    try:
        core.create_namespaced_service_account(namespace, sa_body)
    except ApiException as e:
        if e.status == 409:
            core.patch_namespaced_service_account(sa_name, namespace, sa_body)
        else:
            raise

    # ---- InferenceService (v1beta1) 정의 ----
    group, version, plural = "serving.kserve.io", "v1beta1", "inferenceservices"
    annotations_isvc = {
        "autoscaling.knative.dev/minScale": str(min_replicas),
        "autoscaling.knative.dev/maxScale": str(max_replicas),
        # 일부 설치에서 메타에 s3 힌트를 같이 주면 더 안정적
        "serving.kserve.io/s3-endpoint":  s3_host,
        "serving.kserve.io/s3-usehttps":  "1",
        "serving.kserve.io/s3-verifyssl": "1",
        "serving.kserve.io/s3-region":    s3_region,
    }

    if use_legacy_spec:
        predictor = {
            "serviceAccountName": sa_name,
            "sklearn": {
                "storageUri": storage_uri,
                "resources": {
                    "requests": {"cpu": "200m", "memory": "512Mi"},
                    "limits":   {"cpu": "1",    "memory": "2Gi"},
                },
            },
        }
    else:
        predictor = {
            "serviceAccountName": sa_name,
            "model": {
                "modelFormat": {"name": "sklearn"},
                "storageUri": storage_uri,
                "resources": {
                    "requests": {"cpu": "200m", "memory": "512Mi"},
                    "limits":   {"cpu": "1",    "memory": "2Gi"},
                },
            },
        }

    isvc_body = {
        "apiVersion": f"{group}/{version}",
        "kind": "InferenceService",
        "metadata": {
            "name": isvc_name,
            "namespace": namespace,
            "annotations": annotations_isvc,
        },
        "spec": {"predictor": predictor},
    }

    # ---- 생성 또는 패치 ----
    created = False
    try:
        crd.create_namespaced_custom_object(group, version, namespace, plural, isvc_body)
        created = True
    except ApiException as e:
        if e.status == 409:
            crd.patch_namespaced_custom_object(group, version, namespace, plural, isvc_name, isvc_body)
        else:
            raise

    # ---- Ready 대기 (최대 3분) ----
    url, ready = "", ""
    deadline = time.time() + 180
    while time.time() < deadline:
        obj = crd.get_namespaced_custom_object(group, version, namespace, plural, isvc_name)
        status = obj.get("status", {})
        conds = {c.get("type"): c for c in status.get("conditions", [])}
        if "Ready" in conds:
            ready = conds["Ready"].get("status", "")
        url = status.get("url") or status.get("address", {}).get("url") or ""
        if ready == "True" and url:
            break
        time.sleep(3)

    # ---- 결과 산출물 ----
    with open(isvc_name_out, "w", encoding="utf-8") as f:
        f.write(isvc_name)
    with open(isvc_yaml_out, "w", encoding="utf-8") as f:
        yaml.safe_dump(isvc_body, f, sort_keys=False, allow_unicode=True)
    with open(isvc_url_out, "w", encoding="utf-8") as f:
        f.write(url or "")
    with open(log_txt, "w", encoding="utf-8") as f:
        f.write(
            "=== KServe Deploy Result ===\n"
            f"namespace : {namespace}\n"
            f"isvc_name : {isvc_name}\n"
            f"created   : {created}\n"
            f"ready     : {ready}\n"
            f"url       : {url}\n"
            f"storageUri: {storage_uri}\n"
            f"sa/secret : {sa_name} / {secret_name}\n"
        )

print("[done]")

## 14) 파이프라인 정의
- 컴포넌트들을 연결하여 파이프라인을 정의하고 컴파일합니다.

In [ ]:
# Cell 14 — Pipeline (no visualization; ensure upload runs AFTER tuning/eval)

from kfp import dsl
from kfp import kubernetes as k8s

# 이미 pipeline2가 있다면 재정의 방지
try:
    pipeline2
except NameError:
    from inspect import signature, _empty
    def _eval_annotations(fn):
        sig = signature(fn)
        fixed = {}
        for name, param in sig.parameters.items():
            ann = param.annotation
            if ann is _empty:
                continue
            if isinstance(ann, str) and ann in {"str", "int", "float", "bool"}:
                ann = eval(ann)
            fixed[name] = ann
        if fixed:
            fn.__annotations__.update(fixed)
        return fn

    def pipeline2(**kwargs):
        def _decorator(fn):
            fn = _eval_annotations(fn)
            return dsl.pipeline(**kwargs)(fn)
        return _decorator

@pipeline2(
    name="prelogin-gender-pipeline",
    description="Build features, train baseline, fetch Katib HPs, retrain, tune threshold, upload to Kakao Object Storage, and deploy on KServe.",
)
def prelogin_gender_pipeline(
    # 입력 CSV (PVC로 마운트)
    input_path: str = "/home/jovyan/datasets/gender/processed_user_behavior.joined.csv",

    # 공통 파라미터
    known_cats_json: str = '["Books","Electronics","Gaming","Home","Fashion"]',
    production_threshold: float = 0.5,
    min_prelogin_events: int = 2,

    # 디렉토리 힌트(로컬 경로, 실제 I/O는 아티팩트가 담당)
    datasets_dir: str = "datasets/gender",
    artifact_dir: str = "models/gender",

    # 분할/캘리브레이션
    random_state: int = 42,
    test_size: float = 0.2,
    calib_method: str = "isotonic",
    calib_cv: int = 5,

    # Katib
    katib_namespace: str = "kbm-u-kubeflow-tutorial",
    katib_experiment: str = "gender-logistic-random",

    # 임계값 탐색
    tune_t_lo: float = 0.2,
    tune_t_hi: float = 0.8,
    tune_t_steps: int = 61,

    # PVC 이름
    pvc_name: str = "cpu-notebook-workspace",

    # Kakao Object Storage 업로드(파라미터로 키 전달)
    kakao_endpoint: str = "https://objectstorage.kr-central-2.kakaocloud.com",
    kakao_region: str = "kr-central-2",
    kakao_bucket: str = "models",
    kakao_prefix: str = "gender_predict_pipeline",
    kakao_public_read: bool = False,
    kakao_access_key: str = "",   # ← KFP UI에서 입력
    kakao_secret_key: str = "",   # ← KFP UI에서 입력
):
    # 1) 초기화
    c1 = cell1_init(
        known_cats_json=known_cats_json,
        production_threshold=production_threshold,
        min_prelogin_events=min_prelogin_events,
        datasets_dir=datasets_dir,
        artifact_dir=artifact_dir,
    )

    # 2) CSV 읽기 + prelogin 필터 (PVC 필요)
    c2 = cell2_prelogin_filter(
        input_path=input_path,
        min_prelogin_events=min_prelogin_events,
    )
    k8s.mount_pvc(task=c2, pvc_name=pvc_name, mount_path="/home/jovyan")

    # 3) 숫자 집계 + 라벨 생성
    c3 = cell3_aggregate_and_label(
        df_in=c2.outputs["df_out"],
        pre_in=c2.outputs["pre_out"],
    )

    # 4) 카테고리 파생
    c4 = cell4_category_features(
        pre_in=c2.outputs["pre_out"],
        Xnum_in=c3.outputs["Xnum_out"],
        known_cats_json=known_cats_json,
    )

    # 5) 데이터셋 조립
    c5 = cell5_build_dataset(
        Xnum_in=c3.outputs["Xnum_out"],
        cat_cnt_in=c4.outputs["cat_cnt_out"],
        cat_prop_in=c4.outputs["cat_prop_out"],
        cat_log_in=c4.outputs["cat_log_out"],
        y_in=c3.outputs["y_out"],
    )

    # 6) 분할
    c6 = cell6_split(
        dataset_pq_in=c5.outputs["dataset_pq_out"],
        random_state=random_state,
        test_size=test_size,
    )

    # 7) 기본 로지스틱 + 캘리브레이션 (베이스라인)
    c7 = cell7_train_calibrate(
        Xtr_in=c6.outputs["Xtr_out"],
        ytr_in=c6.outputs["ytr_out"],
        calib_method=calib_method,
        calib_cv=calib_cv,
    )

    # 8) 프로덕션 임계값 평가 (베이스라인 모델)
    c8 = cell8_eval_prod_threshold(
        cal_model_in=c7.outputs["cal_model_out"],
        Xva_in=c6.outputs["Xva_out"],
        yva_in=c6.outputs["yva_out"],
        dataset_csv_in=c5.outputs["dataset_csv_out"],
        production_threshold=production_threshold,
    )

    # 9) Katib HP 가져오기
    c9 = cell9_fetch_katib(
        katib_namespace=katib_namespace,
        katib_experiment=katib_experiment,
    )

    # 10) Katib HP로 재학습 (필요시 원본 CSV 재빌드) → PVC 필요
    c10 = cell10_train_with_katib(
        katib_params_json_in=c9.outputs["katib_params_json_out"],
        input_path=input_path,
        known_cats_json=known_cats_json,
        base_min_prelogin_events=min_prelogin_events,
        Xva_prev_in=c6.outputs["Xva_out"],
        yva_prev_in=c6.outputs["yva_out"],
        dataset_prev_in=c5.outputs["dataset_csv_out"],
    )
    k8s.mount_pvc(task=c10, pvc_name=pvc_name, mount_path="/home/jovyan")

    # 11) 임계값 튜닝 + 메타/확률/연령대 메트릭 저장 (재학습 모델 기준)
    c11 = cell11_tune_threshold_and_save(
        cal_model_in=c10.outputs["cal_model_out"],
        Xva_in=c10.outputs["Xva_out"],
        yva_in=c10.outputs["yva_out"],
        dataset_in=c10.outputs["dataset_out"],
        df_in=c2.outputs["df_out"],
        t_lo=tune_t_lo,
        t_hi=tune_t_hi,
        t_steps=tune_t_steps,
        artifact_dir_hint=artifact_dir,
    )

    # 12) 튜닝/평가 이후 업로드 보장 (재학습 모델 업로드)
    c12 = cell12_upload_model_to_kakao(
        cal_model_in=c10.outputs["cal_model_out"],
        endpoint=kakao_endpoint,
        region=kakao_region,
        bucket=kakao_bucket,
        access_key=kakao_access_key,   # 파라미터로 전달
        secret_key=kakao_secret_key,   # 파라미터로 전달
        prefix=kakao_prefix,
        object_name="model.joblib",
        public_read=kakao_public_read,
    )
    c12.after(c11)  # 반드시 튜닝/평가 후 업로드

    # 13) KServe 배포 — 업로드 완료 후 수행
    c13 = cell13_deploy_kserve(
        namespace="kbm-u-kubeflow-tutorial",
        isvc_name="gender-sklearn",
        bucket=kakao_bucket,            
        prefix=kakao_prefix,               
        s3_endpoint=kakao_endpoint,
        s3_region=kakao_region,
        aws_access_key_id=kakao_access_key,       # 파이프라인 파라미터로 전달
        aws_secret_access_key=kakao_secret_key,   # 파이프라인 파라미터로 전달
        min_replicas=1,
        max_replicas=1,
        use_legacy_spec=False,               # 새 스펙(predictor.model)
        append_model_filename=True,          # 업로드 파일명이 model.joblib이므로 파일까지 붙여 배포
    )
    c13.after(c12)

    # 캐싱 비활성화 (선택)
    for t in [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13]:
        t.set_caching_options(enable_caching=False)

from kfp import compiler
compiler.Compiler().compile(prelogin_gender_pipeline, "gender_pipeline.yaml")
print("[Done] gender_pipeline.yaml Created")


## 15) 모델 추론 실습
- KServe로 배포된 모델에 샘플 데이터를 입력하여 추론하는 실습입니다.

In [ ]:
# Cell 15
import requests, json
import pandas as pd
from IPython.display import display

NS   = "kbm-u-kubeflow-tutorial" # 네임스페이스
ISVC = "gender-sklearn" # KServe InferencesService 이름

GATEWAY = "http://knative-local-gateway.istio-system.svc.cluster.local"

URL = f"{GATEWAY}/v1/models/{ISVC}:predict"

HEADERS = {
    "Host": f"{ISVC}-predictor.{NS}.svc.cluster.local",
    "Content-Type": "application/json",
}

LABEL_MAP = {0: "F", 1: "M"}
TRUTH_MAP = {"여자": "F", "남자": "M"}

#####################
# 10개의 샘플 데이터
#####################
cases = [
    {"case": "29세 남자", "truth": "남자",
     "data": [11,0,0,6.0,1.2727272727272727,7,1,0,3,0,0,1,0,0,0.0,0.0,0.09090909090909091,0.0,0.0,0.0,0.0,0.6931471805599453,0.0,0.0], "note": ""},
    {"case": "68세 여자", "truth": "여자",
     "data": [10,0,0,5.5,1.3,6,0,16,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "56세 여자", "truth": "여자",
     "data": [9,0,0,5.0,1.2222222222222223,6,2,20,2,2,0,0,0,1,0.2222222222222222,0.0,0.0,0.0,0.1111111111111111,1.0986122886681096,0.0,0.0,0.0,0.6931471805599453], "note": ""},
    {"case": "44세 남자", "truth": "남자",
     "data": [9,0,0,5.0,1.5555555555555556,4,0,17,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "28세 여자", "truth": "여자",
     "data": [8,0,0,4.5,1.25,6,1,20,2,0,0,0,1,0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.6931471805599453,0.0], "note": ""},
    {"case": "68세 남자", "truth": "남자",
     "data": [6,0,0,3.5,1.3333333333333333,5,0,22,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], "note": ""},
    {"case": "70세 여자", "truth": "여자",
     "data": [6,0,0,3.5,1.3333333333333333,5,1,19,2,0,0,0,0,1,0.0,0.0,0.0,0.0,0.16666666666666666,0.0,0.0,0.0,0.0,0.6931471805599453], "note": ""},
    {"case": "43세 남자", "truth": "남자",
     "data": [5,0,0,3.0,0.8,4,0,23,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "검색/상품조회/카테고리 접근 없음"},
    {"case": "29세 여자", "truth": "여자",
     "data": [6,0,0,3.5,1.3333333333333333,4,0,2,3,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "남성향 카테고리 활동 위주"},
    {"case": "41세 여자", "truth": "여자",
     "data": [2,0,0,1.5,0.5,2,0,20,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
     "note": "남성향/여성향 고르게 활동"},
]

def infer_one(vec):
    # v1 REST: {"instances": [[features...]]}
    payload = {"instances": [[float(x) for x in vec]]}
    r = requests.post(URL, json=payload, headers=HEADERS, timeout=8)
    r.raise_for_status()
    js = r.json()
    # KServe sklearn predictor(v1) → {"predictions": [int_or_float]}
    if "predictions" not in js:
        raise ValueError(f"Unexpected response keys: {list(js.keys())}")
    return int(js["predictions"][0])

rows = []
for c in cases:
    pred_int = infer_one(c["data"])
    truth_mf = TRUTH_MAP.get(c["truth"], c["truth"])
    pred_mf  = LABEL_MAP.get(pred_int, pred_int)
    rows.append({
        "case":  c["case"],
        "truth": truth_mf,
        "pred":  pred_mf,
        "✓/✗":   "✓" if pred_mf == truth_mf else "✗",
        "note":  c["note"],
    })

df = pd.DataFrame(rows, columns=["case", "truth", "pred", "✓/✗", "note"])
display(df)
acc = (df["✓/✗"] == "✓").mean()
print(f"\n[Done] 정확도(샘플 {len(df)}개): {acc:.3f}")
